<a href="https://colab.research.google.com/github/1pawn0/YTMusic-Concurrent-Playlist-Downloader-via-yt-dlp/blob/main/YTMusic_Concurrent_Playlist_Downloader_via_yt_dlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo add-apt-repository -y ppa:ubuntuhandbook1/ffmpeg8 > /dev/null
!sudo apt -qqy update
!sudo apt -qqy install ffmpeg
!pip install -qU "yt-dlp[default]" bgutil-ytdlp-pot-provider

import sys, os
IN_COLAB = False
if "google.colab" in sys.modules and os.getenv("COLAB_RELEASE_TAG"):
    IN_COLAB = True
    from google.colab import drive
    drive.mount("/content/drive")
    print("Running as a Google Colab notebook \N{CHECK MARK}")

In [ ]:
# @title Youtube Music Playlist Downloader {"form-width":"50%"}
import yt_dlp
import shutil
import concurrent.futures
from pathlib import Path


# Enter the playlist URL here:
PLAYLIST_URL: str = "https://music.youtube.com/playlist?list=PLAYLIST-ID"  # @param PLAYLIST_URL {"type":"string","placeholder":"https://music.youtube.com/playlist?list=PLAYLIST-ID"}


# Enter the directory path to save the downloaded playlist:
DOWNLOAD_PATH: str = "/content/drive/MyDrive/music/yt-music/ytmusic-playlist/"  # @param DOWNLOAD_PATH {"type":"string","placeholder":"/path/to/save/downloaded/playlist"}


def get_playlist_info(playlist_url: str = PLAYLIST_URL) -> dict:
    ydl_opts = {
        "extract_flat": True,  # do NOT process url_results further. Here `extract_flat` MUST be `True` for efficiency.
        "quiet": True,
        "simulate": True,
        "skip_download": True,
        "restrictfilenames": True,
        "fixup": "detect_or_warn",
        # "daterange": yt_dlp.utils.DateRange('20240101','20250901'),
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info: dict = ydl.extract_info(playlist_url, download=False)
        return info


def download_playlist_nth_item(
    pl_item_number: int,
    playlist_url: str = PLAYLIST_URL,
    dl_path: Path = Path(DOWNLOAD_PATH),
    output_template: str = "%(title)s.%(ext)s",
):
    ydl_opts: dict = {
        # "cookiefile": "/content/cookies.txt",  # `cookies.txt` file can be generated by `Get cookies.txt LOCALLY` extension for Chrome
        "playlist_items": str(pl_item_number),
        "outtmpl": output_template,
        "format": "bestaudio",  # At first set "bestaudio" and if it doesn't work, set "bestaudio/best" value for the "format" option
        "format_sort": ["hasaud", "quality", "asr", "abr"],
        "postprocessors": [
            {
                "key": "FFmpegExtractAudio",
                "preferredcodec": "mp3",
                "preferredquality": "0",
            }
        ],
        "paths": {
            "home": dl_path.__str__(),
            "temp": (dl_path / "__temp__").__str__(),
        },
        "restrictfilenames": True,
        "overwrites": False,
        "no_overwrites": True,
        "nooverwrites": True,
        "fixup": "detect_or_warn",
        "ignoreerrors": "only_download",
        # "daterange": yt_dlp.utils.DateRange('20240101','20250901'),
        # "playlistreverse": True,
        # "extract_flat": False,
        # "verbose": False,
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([playlist_url])


def make_zip_archive(dir_path: Path = Path(DOWNLOAD_PATH)) -> Path:
    if not dir_path.is_dir():
        raise ValueError(f"The provided path '{dir_path}' is not a valid directory.")
    archive_path: Path = Path(shutil.make_archive(base_name=dir_path, format="zip", root_dir=dir_path, verbose=True))
    return archive_path


def download_playlist_concurrently(max_workers: int = 32) -> Path:
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        executor.map(download_playlist_nth_item, range(1, get_playlist_info()["playlist_count"] + 1))
    return make_zip_archive()


download_playlist_concurrently()
